In [80]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
from sklearn import linear_model, svm,model_selection
import sklearn
from arftools import *
import numpy as np
import matplotlib.pyplot as plt
from StringKernel import StringKernel
from cross_validation import validation_croisee

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import nltk

In [4]:
data = pd.read_json('data/smaller_dataset.json')

In [35]:
def generate_sample(authors):
    authors_sample = authors['reviewerID'].sample(2)
    subset = authors[authors['reviewerID'].isin(authors_sample)]
    X = subset['reviewText']
    Y = subset['reviewerID']
    vocab = get_vocab(X)
    return (X,Y, vocab)

In [42]:
authors = data[data.groupby(by='reviewerID')['reviewerID'].transform("count") == 20]
samples = [generate_sample(authors) for _ in range(3)]

In [40]:
samples[0][1].shape

(50,)

In [7]:
subset = authors[authors['reviewerID'].isin(authors_sample)]
X = subset['reviewText']
Y = subset['reviewerID']
X_train = X[0:20]
Y_train = Y[0:20]
X_test = X[20:]
Y_test = Y[20:]

In [22]:
def train_and_test(X_train, Y_train, X_test, vocab, lambda_param):
    string_kernel = StringKernel().build_kernel(vocab, lambda_param)
    precomputed_gram_matrix = string_kernel(X_train.values,X_train.values)
    svm_instance = sklearn.svm.SVC(kernel="precomputed")
    svm_instance.fit(precomputed_gram_matrix,Y_train)

    return svm_instance.predict(string_kernel(X_test.values,X_test.values))

def get_vocab(X):
    vocab = set()
    for text in X:
        for word in nltk.word_tokenize(text):
            vocab.add(word)
    return vocab

In [43]:
N = 20
results = []
lambdas = [0.1,0.3,0.5,0.7,0.9,0.95,1]
for i,l in enumerate(lambdas):
    print('lambda : ' + str(l))
    res = []
    for X,Y,vocab in samples:
        prediction = train_and_test(X[0:N], Y[0:N], X[N:], vocab, l)
        prec = (prediction == Y[N:]).sum() / N
        res.append(prec)
    mean = np.array(res).mean()
    print(mean)
    results.append(mean)

lambda : 0.1
0.633333333333
lambda : 0.3
0.633333333333
lambda : 0.5
0.633333333333
lambda : 0.7
0.633333333333
lambda : 0.9
0.633333333333
lambda : 0.95
0.633333333333
lambda : 1
0.633333333333


In [13]:
Y_train

1032061     AJYGQV81FSFE2
1061565     AJYGQV81FSFE2
1089083    A1K94LXX833JTT
1134271     AJYGQV81FSFE2
1150190    A1K94LXX833JTT
1165293    A1K94LXX833JTT
1212022     AJYGQV81FSFE2
1315945     AJYGQV81FSFE2
138297     A1K94LXX833JTT
140449     A1K94LXX833JTT
1432245    A1K94LXX833JTT
171198      AJYGQV81FSFE2
203787     A1K94LXX833JTT
218007     A1K94LXX833JTT
26597      A1K94LXX833JTT
320003      AJYGQV81FSFE2
321596     A1K94LXX833JTT
336274      AJYGQV81FSFE2
49304      A1K94LXX833JTT
531373      AJYGQV81FSFE2
Name: reviewerID, dtype: object

In [14]:
from collections import Counter

In [91]:
kernel = StringKernel(lambda_param=1)
kernel.K_hat('car','cat')

/Users/samutamminen/Documents/Study/Sorbonne/S2/ARF/TME/TME5/StringKernel.py:53: RuntimeWarning: invalid value encountered in double_scalars
  return self.K(self.k, s,t) / np.sqrt(self.K(self.k, s,s) * self.K(self.k, t,t))


nan